In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from utils.transformations import ExtendedTransformation, SimpleTransformation
from utils.filters import SimpleFilter
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("data/preprocessed/train_data.csv")
X_train, y_train = df_train.drop(columns=['Price']), df_train[['Price']]
preprocessor = ExtendedTransformation()
filter = SimpleFilter()
preprocessor.fit(X_train, y_train)
X_processed, y_processed = preprocessor.transform(X_train, y_train)
filter.fit(X_processed, y_processed)
X_filtered, y_filtered = filter.transform(X_processed, y_processed)

X shape:  (20974, 40)
bin_vars_columns shape:  (36,)
low_card_columns shape:  37
X shape:  (20974, 40)
X_low_card   shape:  (20974, 113)
X_high_card shape:  (20974, 50)
X_crossed_features shape:  (20974, 6670)
X_EXPANDED shape:  (20974, 6835)
(20974, 6835)
(20974, 4173)
(20974, 3193)
(20974, 1635)
(20974, 4173)
(20974, 3193)
(20974, 1635)


In [3]:
df_test = pd.read_csv("data/preprocessed/test_data.csv")
X_test, y_test = df_test.drop(columns=['Price']), df_test[['Price']]
X_test_proccesed, y_test_proccessed = preprocessor.transform(X_test, y_test)
X_test_filtered, y_test_filtered = filter.transform(X_test_proccesed, y_test_proccessed)

X shape:  (8989, 40)
X_low_card   shape:  (8989, 113)
X_high_card shape:  (8989, 50)
X_crossed_features shape:  (8989, 6670)
X_EXPANDED shape:  (8989, 6835)
(8989, 4173)
(8989, 3193)
(8989, 1635)


In [16]:
def create_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 6)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for layer in range(n_layers):
        n_neuros = trial.suggest_categorical(f'neurons_of_layer{layer}', [32,64,128,256,512])
        model.add(tf.keras.layers.Dense(n_neuros, activation='relu'))
        dropout_ratio = trial.suggest_float(f'dropout_of_layer{layer}', 0.0, 0.35)
        model.add(tf.keras.layers.Dropout(dropout_ratio))
    model.add(tf.keras.layers.Dense(1))
    return model

In [17]:
def objective(trial):
    X,y = X_filtered, y_filtered
    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3, random_state=42)
    learning_rate = trial.suggest_float("learnin_rate", 0.0001, 0.1, log=True)
    model = create_model(trial)
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(loss='mse', metrics=['mae'], optimizer=optimizer)
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Monitoriza la pérdida en el conjunto de validación
        patience=10,         # Número de épocas sin mejora antes de detener el entrenamient
        verbose=1,           # Muestra mensajes cuando se detiene el entrenamiento
        restore_best_weights=True  # Restaura los pesos del modelo de la mejor época encontrada
        )
    batch_size_param = trial.suggest_categorical('batch_size',[8,16,32,124])
    history = model.fit(X_train, y_train,
                        epochs=200, 
                        batch_size=batch_size_param,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping_callback], verbose=0)
    
    results = model.evaluate(X_val, y_val, verbose=0)
    return results[0], model


In [18]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
print(study.best_trial)

[I 2025-03-27 19:02:40,467] A new study created in memory with name: no-name-981d69d8-93b4-46fb-9162-054b916261f8


Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.


[W 2025-03-27 19:03:08,433] Trial 0 failed with parameters: {'learnin_rate': 0.07153815154240704, 'n_layers': 4, 'neurons_of_layer0': 64, 'dropout_of_layer0': 0.09441626471780273, 'neurons_of_layer1': 256, 'dropout_of_layer1': 0.12368310056399408, 'neurons_of_layer2': 512, 'dropout_of_layer2': 0.12442538481834291, 'neurons_of_layer3': 32, 'dropout_of_layer3': 0.2986738858993689, 'batch_size': 124} because of the following error: The value <Sequential name=sequential_13, built=True> could not be cast to float.
[W 2025-03-27 19:03:08,433] Trial 0 failed with value (0.9979563355445862, <Sequential name=sequential_13, built=True>).


Epoch 24: early stopping
Restoring model weights from the end of the best epoch: 14.


[W 2025-03-27 19:05:35,523] Trial 1 failed with parameters: {'learnin_rate': 0.0027372893897031902, 'n_layers': 5, 'neurons_of_layer0': 64, 'dropout_of_layer0': 0.1350467844488989, 'neurons_of_layer1': 512, 'dropout_of_layer1': 0.02689732102324603, 'neurons_of_layer2': 512, 'dropout_of_layer2': 0.028579360754969387, 'neurons_of_layer3': 32, 'dropout_of_layer3': 0.3014452468569863, 'neurons_of_layer4': 128, 'dropout_of_layer4': 0.1197322740369365, 'batch_size': 8} because of the following error: The value <Sequential name=sequential_14, built=True> could not be cast to float.
[W 2025-03-27 19:05:35,524] Trial 1 failed with value (0.6552473902702332, <Sequential name=sequential_14, built=True>).


...

Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 10.


[W 2025-03-27 20:12:06,966] Trial 48 failed with parameters: {'learnin_rate': 0.009910206253590605, 'n_layers': 4, 'neurons_of_layer0': 128, 'dropout_of_layer0': 0.16815179747245346, 'neurons_of_layer1': 32, 'dropout_of_layer1': 0.03238507776875205, 'neurons_of_layer2': 64, 'dropout_of_layer2': 0.2870415646041326, 'neurons_of_layer3': 512, 'dropout_of_layer3': 0.15978425358583148, 'batch_size': 32} because of the following error: The value <Sequential name=sequential_61, built=True> could not be cast to float.
[W 2025-03-27 20:12:06,966] Trial 48 failed with value (0.6451274752616882, <Sequential name=sequential_61, built=True>).


Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 19.


[W 2025-03-27 20:12:42,322] Trial 49 failed with parameters: {'learnin_rate': 0.0045990284462273726, 'n_layers': 3, 'neurons_of_layer0': 128, 'dropout_of_layer0': 0.13300770183488833, 'neurons_of_layer1': 512, 'dropout_of_layer1': 0.10941498267280986, 'neurons_of_layer2': 32, 'dropout_of_layer2': 0.2919188383190103, 'batch_size': 32} because of the following error: The value <Sequential name=sequential_62, built=True> could not be cast to float.
[W 2025-03-27 20:12:42,322] Trial 49 failed with value (0.5549209117889404, <Sequential name=sequential_62, built=True>).


ValueError: No trials are completed yet.